In [1]:
from db import *
import re

In [2]:
def remove_punct(lk):
    lk1=[]
    for i in lk.lower().split():
        l = (i.encode("ascii", "ignore")).decode("utf-8")
        if "\\" not in l:
            lk1.append(l)
    lk1 = " ".join(lk1)
    punctuations = '''!()-[]{};:'"\<>.,=/?@$%^&*_~|'''
    lk2 = ""
    for char in lk1:
        if char not in punctuations:
            lk2 = lk2 + char
        else:
            lk2 = lk2 + " "

    return re.sub(" +", " ", lk2)

In [3]:
def soft_skils_checker(descrption_data):
    soft_skills=[]
    l=[]
    generic_skills = skillsets_coll.find({"skill_type": "generic"})
    for i in generic_skills:
        skill=" "+remove_punct(i["name"])+" "
        if descrption_data.count(skill)>=1:
            kls=remove_punct(i["name"])
            if kls not in l:
                l.append(kls)
                soft_skills.append([kls,i["discernablity"]])
    return soft_skills


In [4]:
def techncial_skill_checker(descrption_data):
    hard_skills=[]
    l=[]
    specific_skills = skillsets_coll.find({"skill_type": "specific"})
    for i in specific_skills:
        skill=" "+remove_punct(str(i["name"]))+" "
        if descrption_data.count(skill)>=1:
            kls = remove_punct(i["name"])
            if kls not in l:
                l.append(kls)
                hard_skills.append([kls, i["discernablity"]])
    return hard_skills

In [5]:
def student_skill_matching():
    resumes=student_resumes.find({})
    for resume in resumes:
        resume1=resume["preprocessed"]
        hard_skills=techncial_skill_checker(resume1)
        soft_skills=soft_skils_checker(resume1)
        student_resumes.update_one({"_id":resume["_id"]},{"$set":{"soft_skills":soft_skills,
                                                         "technical_skills":hard_skills}})

In [6]:
def job_skill_matching():
    jobs=aracomo_jobs.find()
    for job in jobs:
        resume1=job["job_descrption"]
        hard_skills=techncial_skill_checker(resume1)
        soft_skills=soft_skils_checker(resume1)
        aracomo_jobs.update_one({"_id":job["_id"]},{"$set":{"soft_skills":soft_skills,
                                                         "technical_skills":hard_skills}})

In [9]:
def job_matching_score():
    jobs=aracomo_jobs.find()
    for job in jobs:
        job_title = job["job_title"]
        job_title = job_title.split()
        job_title = "_".join(job_title)
        resumes = student_resumes.find()
        for resume in resumes:
            technical_skills=[]
            for i in resume["technical_skills"]:
                for j in job["technical_skills"]:
                    if i[0]==j[0]:
                        technical_skills.append([i[0],i[1]])
            #print technical_skills

            soft_skills = []
            for i in resume["soft_skills"]:
                for j in job["soft_skills"]:
                    if i[0] == j[0]:
                        soft_skills.append([i[0],i[1]])
            #print soft_skills
            score=0
            if len(technical_skills)==0:
                score=0
            else:
                for i in technical_skills:
                    score=score+float(i[1])
                score=score+len(soft_skills)

            student_resumes.update_one({"_id": resume["_id"]}, {"$set": {str(job_title)+"soft_skills": soft_skills,
                                                               str(job_title)+"technical_skills": technical_skills,
                                                                     str(job_title)+"score":score}})

In [11]:
student_skill_matching()
job_skill_matching()
job_matching_score()

[[u'analytics', 3.0], [u'driving', 3.0], [u'statistical analysis', 3.0], [u'talent management', 3.0], [u'hr', 7.0]]
[[u'strategy', 3.0], [u'motivation', 3.0], [u'professional development', 3.0], [u'review', 3.0], [u'processes', 3.0], [u'leadership', 3.0], [u'business', 3.0], [u'research', 3.0], [u'organization', 3.0], [u'behavior', 3.0], [u'analysis', 3.0], [u'management', 3.0], [u'readiness', 3.0], [u'disciplined', 3.0], [u'collaboration', 3.0], [u'development', 3.0], [u'reports', 3.0], [u'assessment', 3.0], [u'excel', 3.0]]
[[u'modeling', 3.0], [u'simulation', 3.0], [u'history', 3.0], [u'petrophysics simulation', u'7'], [u'petroleum engineering specialist', u'7'], [u'oil reservoir management', u'7'], [u'geology', u'3'], [u'core and fluid analysis', u'3'], [u'data acquisition', u'3'], [u'strategic surveillance', u'3'], [u'petrophysics', u'3'], [u'reservoir engineering', u'3'], [u'petroleum engineering', u'3'], [u'reservoir management', u'3'], [u'reservoir development', u'7']]
[[u'desi